In [1]:
import sys
import pathlib
import numpy as np
import pandas as pd
sys.path.insert(0, "../0.data-download/scripts/")
from data_loader import load_data
from sklearn.model_selection import train_test_split
import random

In [2]:
random.seed(18)
print(random.random())

0.18126486333322134


In [3]:
# load all of the data 
data_directory = "../0.data-download/data/"
dfs = load_data(data_directory, adult_or_pediatric = "all")
dependency_df = dfs[1]
model_df = dfs[0]

In [5]:
# verifying that the DepMap_IDs in model_df and dependency_df are alligned
verify_df = model_df
verify_df["ID_allignment_verify"] = np.where(dependency_df["DepMap_ID"] == model_df["DepMap_ID"], "True", "False")
verrify = len(verify_df["ID_allignment_verify"].unique())
print(verify_df["ID_allignment_verify"])
print(f"There is {verrify} output object contained in the ID_allignment_verify column \n")

0       True
1       True
2       True
3       True
4       True
        ... 
1081    True
1082    True
1083    True
1084    True
1085    True
Name: ID_allignment_verify, Length: 1086, dtype: object
There is 1 output object contained in the ID_allignment_verify column 



In [7]:
# assign 'age_categories' and 'sex' columns to the dependency dataframe as a single column
presplit_dependency_df = dependency_df.assign(age_and_sex = model_df.age_categories.astype(str) + "_" + model_df.sex.astype(str))
presplit_dependency_df

DepMap_ID  A1BG (1)  A1CF (29974)   A2M (2)  A2ML1 (144568)  \
0     ACH-000001  0.094568      0.012519  0.027460        0.025962   
1     ACH-000004  0.012676      0.049011  0.075933        0.033215   
2     ACH-000005  0.053957      0.027968  0.010139        0.005448   
3     ACH-000007  0.026704      0.083588  0.008853        0.011299   
4     ACH-000009  0.059383      0.051826  0.015370        0.011721   
...          ...       ...           ...       ...             ...   
1081  ACH-002834  0.012720      0.083840  0.013308        0.013315   
1082  ACH-002847  0.058547      0.079576  0.036707        0.030234   
1083  ACH-002874  0.169875      0.017430  0.017965        0.008615   
1084  ACH-002875  0.035519      0.099789  0.071748        0.048479   
1085  ACH-002926  0.022010      0.023496  0.012432        0.007420   

      A3GALT2 (127550)  A4GALT (53947)  A4GNT (51146)  AAAS (8086)  \
0             0.073412        0.027340       0.020199     0.284733   
1             0.013176        0.097497       0.005015     0.153166   
2             0.018599        0.081636       0.005457     0.159904   
3             0.027288        0.028349       0.032573     0.166503   
4             0.030062        0.078373       0.042128     0.184783   
...                ...             ...            ...          ...   
1081          0.107286        0.079063       0.013340     0.717478   
1082          0.069976        0.032828       0.012784     0.476383   
1083          0.117915        0.075693       0.087961     0.144152   
1084          0.245413        0.154663       0.045155     0.324723   
1085          0.018041        0.035598       0.057725     0.272390   

      AACS (65985)  ...  ZW10 (9183)  ZWILCH (55055)  ZWINT (11130)  \
0         0.022084  ...     0.555867        0.037449       0.080585   
1         0.007358  ...     0.007427        0.038768       0.230569   
2         0.050884  ...     0.071035        0.017479       0.274568   
3         0.047045  ...     0.213754        0.196233       0.615338   
4         0.032335  ...     0.169463        0.152385       0.405712   
...            ...  ...          ...             ...            ...   
1081      0.040061  ...     0.493501        0.027170       0.636352   
1082      0.096910  ...     0.731151        0.476423       0.967625   
1083      0.140377  ...     0.462012        0.710398       0.747477   
1084      0.089083  ...     0.090844        0.329219       0.168367   
1085      0.034980  ...     0.605191        0.669718       0.939699   

      ZXDC (79364)  ZYG11A (440590)  ZYG11B (79699)  ZYX (7791)  \
0         0.004241         0.082956        0.012000    0.003592   
1         0.011203         0.060266        0.128375    0.005911   
2         0.033416         0.034712        0.092832    0.012482   
3         0.005432         0.035241        0.138445    0.103161   
4         0.056461         0.189550        0.328064    0.035161   
...            ...              ...             ...         ...   
1081      0.029567         0.029711        0.024132    0.174946   
1082      0.010776         0.072452        0.131375    0.115486   
1083      0.017923         0.077515        0.094231    0.166188   
1084      0.017571         0.101216        0.087739    0.050947   
1085      0.057491         0.028312        0.104833    0.008261   

      ZZEF1 (23140)  ZZZ3 (26009)     age_and_sex  
0          0.012679      0.324623    Adult_Female  
1          0.004645      0.042530      Adult_Male  
2          0.020843      0.050412      Adult_Male  
3          0.146222      0.274833      Adult_Male  
4          0.058402      0.269194      Adult_Male  
...             ...           ...             ...  
1081       0.091510      0.421153  Missing_Female  
1082       0.094022      0.438092    Adult_Female  
1083       0.058117      0.302708    Adult_Female  
1084       0.182081      0.349872      Adult_Male  
1085       0.035919      0.404214  Missing_Female  

[1086 rows x 16710 columns]

In [8]:
groups = model_df.groupby("age_categories")
df_list = []
for name, df in groups:
    
    # only looking for samples that contain Adult or Pediatric information
    if name == "Adult" or name == "Pediatric":
        df_list.append(df)
        
# merge sample dataframes through concatentation and reorganize so that DepMap_IDs are in alphabetical order
new_df = pd.concat(df_list, axis=0)
new_df = new_df.set_index("DepMap_ID")
new_df = new_df.sort_index(ascending=True)
new_df = new_df.reset_index()

In [9]:
# creating a list of DepMap_IDs that correlate to pediatric and adult samples
PA_dependency_IDs = new_df["DepMap_ID"].tolist()

PA_IDs = set(PA_dependency_IDs) & set(presplit_dependency_df["DepMap_ID"].tolist())

# creating a new gene dependency data frame containing correlating DepMap_IDs to the filtered sample info IDs
PA_dependency_df = presplit_dependency_df.loc[presplit_dependency_df["DepMap_ID"].isin(PA_IDs)].reset_index(drop=True)

In [10]:
#split the data based on age category and sex
train_df, test_df = train_test_split(
    PA_dependency_df, 
    test_size = .15, 
    stratify= PA_dependency_df.age_and_sex
)

In [11]:
# save the TESTING dataframe 
test_df = test_df.reset_index(drop=True)
testing_df_output = pathlib.Path("../0.data-download/data/VAE_test_df.csv")
test_df.to_csv(testing_df_output, index = False)
print(test_df.shape)
test_df.head(3)

(133, 16710)


DepMap_ID  A1BG (1)  A1CF (29974)   A2M (2)  A2ML1 (144568)  \
0  ACH-000459  0.012629      0.016391  0.027969        0.034453   
1  ACH-000793  0.126993      0.015702  0.025510        0.058065   
2  ACH-000477  0.009234      0.023683  0.066097        0.074418   

   A3GALT2 (127550)  A4GALT (53947)  A4GNT (51146)  AAAS (8086)  AACS (65985)  \
0          0.182521        0.018611       0.047235     0.069901      0.028130   
1          0.181468        0.120454       0.012872     0.447099      0.060567   
2          0.061956        0.034797       0.006806     0.078507      0.076856   

   ...  ZW10 (9183)  ZWILCH (55055)  ZWINT (11130)  ZXDC (79364)  \
0  ...     0.025934        0.016659       0.053245      0.002403   
1  ...     0.147287        0.500022       0.869618      0.015395   
2  ...     0.057472        0.017496       0.295494      0.149025   

   ZYG11A (440590)  ZYG11B (79699)  ZYX (7791)  ZZEF1 (23140)  ZZZ3 (26009)  \
0         0.081088        0.161222    0.018495       0.019509      0.045409   
1         0.069857        0.076031    0.045482       0.120410      0.347476   
2         0.077618        0.010605    0.014924       0.008286      0.424595   

   age_and_sex  
0   Adult_Male  
1   Adult_Male  
2   Adult_Male  

[3 rows x 16710 columns]

In [12]:
# save the TRAINING dataframe 
train_df = train_df.reset_index(drop=True)
training_df_output = pathlib.Path("../0.data-download/data/VAE_train_df.csv")
train_df.to_csv(training_df_output, index = False)
print(train_df.shape)
train_df.head(3)

(751, 16710)


DepMap_ID  A1BG (1)  A1CF (29974)   A2M (2)  A2ML1 (144568)  \
0  ACH-000822  0.037779      0.035940  0.014023        0.014884   
1  ACH-000054  0.121542      0.033057  0.034461        0.023655   
2  ACH-000115  0.166098      0.026557  0.056788        0.019573   

   A3GALT2 (127550)  A4GALT (53947)  A4GNT (51146)  AAAS (8086)  AACS (65985)  \
0          0.093606        0.046031       0.015361     0.336440      0.044045   
1          0.067816        0.030282       0.033054     0.013902      0.027737   
2          0.127481        0.053332       0.015325     0.038566      0.043611   

   ...  ZW10 (9183)  ZWILCH (55055)  ZWINT (11130)  ZXDC (79364)  \
0  ...     0.199975        0.075425       0.930082      0.027703   
1  ...     0.618865        0.126937       0.877066      0.022854   
2  ...     0.301632        0.022844       0.583423      0.114499   

   ZYG11A (440590)  ZYG11B (79699)  ZYX (7791)  ZZEF1 (23140)  ZZZ3 (26009)  \
0         0.044858        0.209492    0.040384       0.058936      0.281196   
1         0.089039        0.062127    0.037931       0.037812      0.291740   
2         0.087140        0.048662    0.035922       0.304547      0.363692   

   age_and_sex  
0   Adult_Male  
1   Adult_Male  
2   Adult_Male  

[3 rows x 16710 columns]